# Necessary Tools and Services

In [2]:
! pip install pandas
! pip install matplotlib
! pip install numpy
! pip install seaborn
! pip install unzip
! pip install gensim
! pip install nltk
! pip install wordcloud
! pip install spacy
! pip install spacy_download
! pip install pyLDAvis
! pip install PyStemmer

! python3 -m spacy download en

  Using cached matplotlib-3.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached fonttools-4.41.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.5 MB)
  Using cached contourpy-1.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached pyparsing-3.0.9-py3-none-any.whl (98 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached kiwisolver-1.4.4-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 KB 2.0 MB/s eta 0:00:0000:0100:01
  Using cached unzip-1.0.0-py3-none-any.whl
  Using cached gensim-4.3.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.4 MB)
  Using cached smart_open-6.3.0-py3-none-any.whl (56 kB)
  Using cached wordcloud-1.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (455 kB)
  Using cached spacy-3.6.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.7 MB)
  Usin

In [3]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dev/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/dev/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/dev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.6.0/en_core_web_sm-3.6.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Import data and Preprocess

In [4]:
new_df = pd.read_csv('Dataset/StackOverflowPostsDataset.csv')
new_df["merged"] = new_df[["Body", "Title", "Tags"]].apply("-".join, axis=1)
#new_df.head()
new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = new_df.merged.values.tolist()
data

["<p>When dealing with small projects, what do you feel is the break even point for storing data in simple text files, hash tables, etc., versus using a real database?  For small projects with simple data management requirements, a real database is unnecessary complexity and violates YAGNI.  However, at some point the complexity of a database is obviously worth it.  What are some signs that your problem is too complex for simple ad-hoc techniques and needs a real database?</p>\n\n<p>Note:  To people used to enterprise environments, this will probably sound like a weird question.  However, my problem domain is bioinformatics.  Most of my programming is prototypes, not production code.  I'm primarily a domain expert and secondarily a programmer.  Most of my code is algorithm-centric, not data management-centric.  The purpose of this question is largely for me to figure out how much work I might save in the long run if I learn to use proper databases in my code instead of the more ad-hoc 

In [4]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

print(data[:1])

['When dealing with small projects, what do you feel is the break even point '
 'for storing data in simple text files, hash tables, etc., versus using a '
 'real database? For small projects with simple data management requirements, '
 'a real database is unnecessary complexity and violates YAGNI. However, at '
 'some point the complexity of a database is obviously worth it. What are some '
 'signs that your problem is too complex for simple ad-hoc techniques and '
 'needs a real database? Note: To people used to enterprise environments, this '
 'will probably sound like a weird question. However, my problem domain is '
 'bioinformatics. Most of my programming is prototypes, not production code. '
 'Im primarily a domain expert and secondarily a programmer. Most of my code '
 'is algorithm-centric, not data management-centric. The purpose of this '
 'question is largely for me to figure out how much work I might save in the '
 'long run if I learn to use proper databases in my code in

# Tokenize words and Clean-up text

In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['when', 'dealing', 'with', 'small', 'projects', 'what', 'do', 'you', 'feel', 'is', 'the', 'break', 'even', 'point', 'for', 'storing', 'data', 'in', 'simple', 'text', 'files', 'hash', 'tables', 'etc', 'versus', 'using', 'real', 'database', 'for', 'small', 'projects', 'with', 'simple', 'data', 'management', 'requirements', 'real', 'database', 'is', 'unnecessary', 'complexity', 'and', 'violates', 'yagni', 'however', 'at', 'some', 'point', 'the', 'complexity', 'of', 'database', 'is', 'obviously', 'worth', 'it', 'what', 'are', 'some', 'signs', 'that', 'your', 'problem', 'is', 'too', 'complex', 'for', 'simple', 'ad', 'hoc', 'techniques', 'and', 'needs', 'real', 'database', 'note', 'to', 'people', 'used', 'to', 'enterprise', 'environments', 'this', 'will', 'probably', 'sound', 'like', 'weird', 'question', 'however', 'my', 'problem', 'domain', 'is', 'bioinformatics', 'most', 'of', 'my', 'programming', 'is', 'prototypes', 'not', 'production', 'code', 'im', 'primarily', 'domain', 'expert', 'an

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['deal', 'small', 'project', 'feel', 'break', 'even', 'point', 'store', 'datum', 'simple', 'text', 'file', 'hash', 'table', 'use', 'real', 'database', 'small', 'project', 'simple', 'data', 'management', 'requirement', 'real', 'database', 'unnecessary', 'complexity', 'violate', 'however', 'point', 'complexity', 'database', 'obviously', 'worth', 'sign', 'problem', 'complex', 'simple', 'ad_hoc', 'technique', 'need', 'real', 'database', 'note', 'people', 'use', 'enterprise', 'environment', 'probably', 'sound', 'weird', 'question', 'however', 'problem', 'domain', 'bioinformatic', 'programming', 'prototype', 'production', 'code', 'm', 'primarily', 'domain', 'expert', 'secondarily', 'management', 'centric', 'purpose', 'question', 'largely', 'figure', 'much', 'work', 'save', 'long', 'run', 'learn', 'use', 'proper', 'database', 'code', 'instead', 'technique', 'typically', 'use', 'database', 'plain_text']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 2), (7, 1), (8, 6), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 2), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 2), (44, 3), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 3), (51, 2), (52, 1), (53, 1), (54, 1), (55, 2), (56, 1), (57, 1), (58, 1), (59, 4), (60, 1), (61, 1), (62, 1), (63, 1)]]


In [33]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('ad_hoc', 1),
  ('bioinformatic', 1),
  ('break', 1),
  ('centric', 1),
  ('code', 2),
  ('complex', 1),
  ('complexity', 2),
  ('data', 1),
  ('database', 6),
  ('datum', 1),
  ('deal', 1),
  ('domain', 2),
  ('enterprise', 1),
  ('environment', 1),
  ('even', 1),
  ('expert', 1),
  ('feel', 1),
  ('figure', 1),
  ('file', 1),
  ('hash', 1),
  ('however', 2),
  ('instead', 1),
  ('largely', 1),
  ('learn', 1),
  ('long', 1),
  ('m', 1),
  ('management', 2),
  ('much', 1),
  ('need', 1),
  ('note', 1),
  ('obviously', 1),
  ('people', 1),
  ('plain_text', 1),
  ('point', 2),
  ('primarily', 1),
  ('probably', 1),
  ('problem', 2),
  ('production', 1),
  ('programming', 1),
  ('project', 2),
  ('proper', 1),
  ('prototype', 1),
  ('purpose', 1),
  ('question', 2),
  ('real', 3),
  ('requirement', 1),
  ('run', 1),
  ('save', 1),
  ('secondarily', 1),
  ('sign', 1),
  ('simple', 3),
  ('small', 2),
  ('sound', 1),
  ('store', 1),
  ('table', 1),
  ('technique', 2),
  ('text', 1),
  ('

In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [35]:
# Print the Keyword in the 20 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.316*"process" + 0.109*"module" + 0.105*"line" + 0.052*"python" + '
  '0.028*"usr_local" + 0.027*"flag" + 0.026*"node" + 0.026*"scale" + '
  '0.022*"location" + 0.021*"traceback"'),
 (1,
  '0.409*"file" + 0.125*"script" + 0.081*"command" + 0.071*"import" + '
  '0.025*"load" + 0.024*"open" + 0.024*"project" + 0.021*"report" + '
  '0.020*"merge" + 0.016*"copy"'),
 (2,
  '0.144*"table" + 0.104*"teradata" + 0.089*"column" + 0.081*"select" + '
  '0.063*"query" + 0.061*"row" + 0.056*"date" + 0.034*"sql" + 0.025*"insert" + '
  '0.020*"varchar"'),
 (3,
  '0.216*"com" + 0.209*"java" + 0.063*"action" + 0.047*"force" + 0.032*"jdbc" '
  '+ 0.025*"weight" + 0.021*"stack" + 0.021*"play" + 0.020*"mark" + '
  '0.011*"country"'),
 (4,
  '0.147*"error" + 0.130*"run" + 0.083*"job" + 0.041*"user" + 0.039*"log" + '
  '0.038*"test" + 0.038*"base" + 0.037*"execute" + 0.035*"fail" + '
  '0.020*"message"'),
 (5,
  '0.475*"thread" + 0.100*"memory" + 0.076*"convert" + 0.036*"calculate" + '
  '0.010*"uni

In [11]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -14.197674254680775


In [12]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3808045741859483


# Visualize the topics-keywords

In [38]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.261889  0.203259       1        1  32.028058
4      0.231762  0.103536       2        1  10.720898
14     0.157975  0.178567       3        1   9.975133
2      0.085826  0.124792       4        1   7.429707
17     0.243516 -0.285999       5        1   6.898692
1      0.159223 -0.267888       6        1   6.080242
8      0.173097 -0.124605       7        1   4.563010
18     0.032725  0.068190       8        1   3.437431
13    -0.040625  0.034261       9        1   2.968433
10    -0.015850  0.112308      10        1   2.866678
0      0.005868 -0.033696      11        1   2.493841
9     -0.044006  0.001133      12        1   2.412897
7     -0.075330 -0.015445      13        1   2.008157
16    -0.103661  0.001504      14        1   1.870215
3     -0.163973 -0.017500      15        1   1.384398
15    -0.155555 -0.019533      16        1   0.775989
19    -0.159146 -0.008794      17        1   0.767852
5     -0.166144 -0.019595      18        1   0.711485
12    -0.200888 -0.018652      19        1   0.450765
6     -0.226703 -0.015844      20        1   0.156118, topic_info=           Term          Freq         Total Category  logprob  loglift
18         file  17781.000000  17781.000000  Default  30.0000  30.0000
1093       rule  10900.000000  10900.000000  Default  29.0000  29.0000
151       input   9624.000000   9624.000000  Default  28.0000  28.0000
202      output  11176.000000  11176.000000  Default  27.0000  27.0000
9         datum   5283.000000   5283.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
1791      param      0.024451      1.194495  Topic20 -10.4766   2.5735
1688       path      0.024451      1.194377  Topic20 -10.4767   2.5736
2910  snakemake      0.024460      1.198224  Topic20 -10.4763   2.5708
9965        bam      0.024448      1.193572  Topic20 -10.4768   2.5741
8119     docker      0.024448      1.193385  Topic20 -10.4768   2.5743

[804 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
1084      7  0.951397  ability
87        1  0.674291     able
87        3  0.181299     able
87       10  0.143789     able
787      17  0.994315   accept
...     ...       ...      ...
175       5  0.041286    write
175       8  0.058286    write
699      11  0.922676      xml
699      14  0.070975      xml
612       4  0.989906  yyyy_mm

[705 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 5, 15, 3, 18, 2, 9, 19, 14, 11, 1, 10, 8, 17, 4, 16, 20, 6, 13, 7])

In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -11.065073129111033


In [15]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.42665830974002067


In [16]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.277158  0.139558       1        1  27.329699
11    -0.276118  0.045182       2        1  19.661416
9     -0.192942 -0.234792       3        1  11.601036
2     -0.138394  0.189810       4        1   8.897018
1     -0.043632 -0.286090       5        1   5.925223
5     -0.018262 -0.140249       6        1   5.006788
7     -0.050893  0.083484       7        1   4.549239
13    -0.038963  0.098104       8        1   4.175832
4      0.073478  0.037026       9        1   3.261935
8      0.057814 -0.038169      10        1   3.115456
3      0.120515 -0.028487      11        1   2.359182
0      0.146472  0.039965      12        1   1.971950
10     0.165100  0.002387      13        1   1.548905
12     0.235845  0.046080      14        1   0.304118
6      0.237137  0.046191      15        1   0.292203, topic_info=                       Term          Freq         Total Category  logprob  \
18                     file  14097.000000  14097.000000  Default  30.0000   
2910              snakemake   8803.000000   8803.000000  Default  29.0000   
1093                   rule   8613.000000   8613.000000  Default  28.0000   
202                  output   8603.000000   8603.000000  Default  27.0000   
111                  sample   5699.000000   5699.000000  Default  26.0000   
...                     ...           ...           ...      ...      ...   
24307          easyblock_py      0.039703      1.078171  Topic15 -10.6187   
3580   environment_variable      0.039693      1.079653  Topic15 -10.6190   
3248              dataframe      0.039694      1.087974  Topic15 -10.6190   
24341         libjpeg_turbo      0.039681      1.077120  Topic15 -10.6193   
1427                   root      0.039678      1.080093  Topic15 -10.6194   

       loglift  
18     30.0000  
2910   29.0000  
1093   28.0000  
202    27.0000  
111    26.0000  
...        ...  
24307   2.5339  
3580    2.5323  
3248    2.5246  
24341   2.5343  
1427    2.5315  

[641 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
866       2  0.997661         _
787      10  0.994255    accept
64        2  0.785355    access
64        6  0.028118    access
64        7  0.186158    access
...     ...       ...       ...
3494     11  0.999117    worker
1095     10  0.999659  workflow
175       2  0.969050     write
175       3  0.029901     write
699       6  0.991639       xml

[673 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 12, 10, 3, 2, 6, 8, 14, 5, 9, 4, 1, 11, 13, 7])

In [17]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.92758251942754

Coherence Score:  0.4304558381590035


In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.242184  0.165389       1        1  36.092098
8      0.212188 -0.256646       2        1  14.650907
2      0.132557  0.259405       3        1  12.357409
1      0.217190 -0.198196       4        1  10.769402
7      0.193779  0.060637       5        1   9.375639
5      0.003748 -0.073951       6        1   5.023239
0     -0.052638  0.124217       7        1   3.808786
4     -0.139527  0.074625       8        1   2.895835
3     -0.192639 -0.043987       9        1   1.944542
9     -0.151961 -0.072065      10        1   1.895295
10    -0.225389 -0.023189      11        1   0.870393
6     -0.239492 -0.016239      12        1   0.316457, topic_info=                  Term          Freq         Total Category  logprob  loglift
18                file  12004.000000  12004.000000  Default  30.0000  30.0000
2910         snakemake   7703.000000   7703.000000  Default  29.0000  29.0000
1093              rule   7537.000000   7537.000000  Default  28.0000  28.0000
202             output   7553.000000   7553.000000  Default  27.0000  27.0000
344            process   4157.000000   4157.000000  Default  26.0000  26.0000
...                ...           ...           ...      ...      ...      ...
23393          cust_nm      0.047647      0.965564  Topic12 -10.5161   2.7468
20482      entry_point      0.047450      0.964432  Topic12 -10.5202   2.7439
25384        pointplot      0.047972      0.978791  Topic12 -10.5093   2.7400
25577       cdirs_user      0.051701      1.135204  Topic12 -10.4344   2.6667
25585  pertoldi_legend      0.051701      1.135204  Topic12 -10.4344   2.6667

[527 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
87        1  0.999269    able
64        1  0.229549  access
64        3  0.028108  access
64        5  0.741350  access
1052      7  0.989595  accord
...     ...       ...     ...
175       1  0.999595   write
4446      2  0.994842    yaml
324       8  0.995036    year
9138      6  0.997907     yml
3385      4  0.997959     zip

[598 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 9, 3, 2, 8, 6, 1, 5, 4, 10, 11, 7])

In [20]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -8.230815871407843

Coherence Score:  0.4283454164390454


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.265836 -0.115559       1        1  22.343811
8     -0.206402  0.127309       2        1  18.185187
2     -0.133672 -0.192989       3        1  16.091887
7     -0.184859  0.057083       4        1  12.908425
0      0.094685 -0.050125       5        1   7.746939
5      0.131173  0.154267       6        1   7.384442
4     -0.001880  0.259821       7        1   6.642998
9      0.142534 -0.232646       8        1   5.088799
3      0.187845 -0.003381       9        1   2.950956
6      0.236412 -0.003780      10        1   0.656557, topic_info=            Term          Freq         Total Category  logprob  loglift
2910   snakemake   7575.000000   7575.000000  Default  30.0000  30.0000
1093        rule   7412.000000   7412.000000  Default  29.0000  29.0000
18          file  11642.000000  11642.000000  Default  28.0000  28.0000
120        value   3915.000000   3915.000000  Default  27.0000  27.0000
202       output   7514.000000   7514.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
7284      mapper      6.305530      7.182837  Topic10  -6.3605   4.8956
7628        bzip      5.378976      6.257742  Topic10  -6.5195   4.8746
7648  decompress      5.132397      6.010184  Topic10  -6.5664   4.8680
303      library     69.074350    267.534225  Topic10  -3.9668   3.6719
470          lib      8.318413    136.944057  Topic10  -6.0835   2.2248

[488 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
866       6  0.478913              _
866       9  0.519303              _
64        3  0.142695         access
64        4  0.758289         access
64        6  0.099061         access
...     ...       ...            ...
4470      7  0.999371  would_require
3222      8  0.994908        wrapper
4446      7  0.996025           yaml
2473     10  0.972269      yesterday
3385      1  0.998821            zip

[664 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 9, 3, 8, 1, 6, 5, 10, 4, 7])